In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

# Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

# Chargement et traitement des phrases du corpus

In [3]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

Les 3 cellules qui suivent servent à montrer le résultat, mais ne les excécutez pas lorsque vous analysez le corpus entier.
Car lorsque le volume de texte est grand, il vaut mieux utiliser un générateur (comme MySentences ci-dessus) qui économise la RAM en streamant les phrases depuis le disque dur.

In [4]:
%time sentences = [s for s in sentences]

CPU times: user 2min 35s, sys: 3.98 s, total: 2min 39s
Wall time: 2min 42s


In [5]:
len(sentences)

8695925

In [6]:
sentences[123]

['6',
 'et',
 'de',
 'substituer',
 'de',
 'nouvelles',
 'dispositions',
 'a',
 'celles',
 'de',
 'l',
 "'",
 'art',
 '.']

# Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [7]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [8]:
type(bigram_phrases.vocab)

collections.defaultdict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [9]:
len(bigram_phrases.vocab.keys())

5529344

Prenons une clé au hasard :

In [10]:
key_ = list(bigram_phrases.vocab.keys())[1344]
print(key_)

b'prise_le'


Le dictionnaire indique le score de cette coocurrence :

In [11]:
bigram_phrases.vocab[key_]

213

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [12]:
%time bigram_phrases[sentences[267]]

CPU times: user 140 µs, sys: 5 µs, total: 145 µs
Wall time: 147 µs


['la',
 'question',
 'serait',
 'simplifiee',
 ',',
 'si',
 'la',
 'societe_philanthropique',
 'nous',
 'indiquait',
 'de',
 'quelle',
 'maniere',
 'elle',
 'pourvoit',
 'aujourd',
 "'",
 'hui',
 'aux',
 'besoins',
 'des',
 'malheureux',
 'qu',
 "'",
 'elle',
 'entretient',
 '.']

# Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [13]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui converti certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [14]:
%time bigram_phraser[sentences[78]]

CPU times: user 105 µs, sys: 0 ns, total: 105 µs
Wall time: 108 µs


['38',
 ',',
 '875',
 '1',
 ',',
 '875',
 'metres',
 'de',
 'caves',
 ',',
 'a',
 '5',
 'centimes',
 'par',
 'metre',
 'et',
 'demi',
 ',',
 'pendant',
 '311',
 'jours',
 '23',
 ',',
 '325',
 'total',
 'f',
 'r',
 '.']

# Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [15]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [16]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

# Création d'un corpus d'unigrams, bigrams, trigrams

In [17]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [21]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    size=64, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    iter=8 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

Error: Pip module Unable to parse debugpy output, please log an issue with https://github.com/microsoft/vscode-jupyter is required for debugging cells. You will need to install it to debug cells.

## Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

In [20]:
outfile = f"../data/bulletins.model"
model.save(outfile)